In [58]:
class BTree(object):
#tree class init

  class Node(object):
  #make nested class for nodes

    def __init__(self, t):
      # defualt constructor for a new node
      self._t = t # breaks when not private, idk
      self.keys = []
      self.children = []
      self.leaf = True

    def split(self, parent, new_val):
      new_node = self.__class__(self._t) # create new node with half of keys in new

      mid_point = len(self.keys)//2 #determine splitting point
      split_value = self.keys[mid_point]
      parent.add_key(split_value) # push node into parent

      new_node.children = self.children[mid_point + 1:] #assign the new node with the first half of the orig node
      self.children = self.children[:mid_point + 1] # orig node now only has the second half
      new_node.keys = self.keys[mid_point+1:]
      self.keys = self.keys[:mid_point]

      # If the new_node has children, set it as internal node
      if len(new_node.children) > 0:
        new_node.leaf = False

      parent.children = parent.add_child(new_node)
      if new_val < split_value:
        return self
      else:
        return new_node

    def add_key(self, value):
      self.keys.append(value)
      self.keys.sort()

    def add_child(self, new_node):
      i = len(self.children) - 1
      while i >= 0 and self.children[i].keys[0] > new_node.keys[0]:
        i -= 1
      return self.children[:i + 1]+ [new_node] + self.children[i + 1:]


# ----------------------------------------------------------------
# B-tree builder


  def __init__(self, t=2):
    self._t = t
    self.root = self.Node(t)

  def insert(self, new_val):
    node = self.root
    # if node is full (2t-1 keys)
    if len(node.keys) == 2 * node._t - 1:
      new_root = self.Node(self._t) # create new node
      new_root.children.append(self.root) # add empty node to children
      new_root.leaf = False # false since root
      node = node.split(new_root, new_val) # spilt the full node
      self.root = new_root

    
    while not node.leaf:
      i = len(node.keys) - 1
      while i > 0 and new_val < node.keys[i] :
        i -= 1
      if new_val > node.keys[i]:
        i += 1

      next = node.children[i]
      if len(next.keys) == 2 * node._t - 1: # most likely thing that broke
        node = next.split(node, new_val)
      else:
        node = next
    # Since we split all full nodes on the way down, we can simply insert the new_val in the leaf.
    node.add_key(new_val)

  def search(self, value, node=None):
    """Return True if the B-Tree contains a key that matches the value."""
    if node is None:
      node = self.root
    # check if value is in key
    if value in node.keys:
      return node.keys
    elif node.leaf:
      return "NULL" #idk i was thinking of putting None 
    # if not in keys, check if value can be in leaf
    else:
      i = 0
      while i < len(node.keys)//2 and value > node.keys[i]:
        i += 1
        # recursively call unto find and return node or return None ("NULL")
      return self.search(value, node.children[i])

  def print_order(self):
    this_level = [self.root]
    while this_level:
      next_level = []
      output = ""
      for node in this_level:
        if node.children:
          next_level.extend(node.children)
        output += str(node.keys) + " "
      print(output)
      this_level = next_level

  def delete(self, value):
    

In [59]:
test = BTree()

for i in range(1,9):
    test.insert(i)
test.print_order()
test.search(11)

[2, 4, 6] 
[1] [3] [5] [7, 8] 


'NULL'